# RAG Chatbot using Gemini

In this notebook, we implement a **Retrieval-Augmented Generation (RAG)** pipeline using a **vector database** built from a file as the knowledge source. information.
<br>
<br>
We will:

- Process and embed text data from the file.
- Store and index embeddings in a vector database for efficient retrieval.
- Retrieve relevant context based on user queries.
- Use Gemini API to generate responses enriched with retrieved knowledge.
<br>
<br>

This approach enables context-aware, accurate, and knowledge-grounded responses, making the model more effective for domain-specific tasks.




## Libraries

In [1]:
# Installing the gen ai package for calling Gemini
!pip install -q -U google-genai langchain langchain-google-genai faiss-cpu langchain-community

In [2]:
from google import genai
from google.colab import userdata
from IPython.display import Markdown
import pandas as pd
import numpy as np
import time
from itertools import islice
from tqdm import tqdm

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4

## Setting up the API Key

For this notebook, I've used my free personal Gemini API key I've stored in Colab's environment variable `gemini`

In [3]:
# Access the API key
gemini_api_key = userdata.get('gemini')

In [4]:
# Configuring the LLM in the package with the API key
client = genai.Client(api_key=gemini_api_key)

## Testing the Use of Gemini through an API

In [5]:
response = client.models.generate_content(
  model='gemini-2.0-flash', # Choosing the LLM model you want to access (limited choices for free tier)
  contents='Who is the GOAT in the NBA? Why?', # Prompt the model
)

Markdown(response.text) # Print the output in markdown format

The "GOAT" (Greatest Of All Time) debate in the NBA is one of the most enduring and passionate topics in sports. There's no single, universally agreed-upon answer, as different people value different aspects of the game.  However, the two names that consistently dominate the discussion are **Michael Jordan** and **LeBron James**.

Here's a breakdown of why each is considered the GOAT, along with some counterarguments:

**Michael Jordan:**

*   **Arguments FOR being the GOAT:**

    *   **Unmatched Dominance:** Jordan led the Chicago Bulls to **two three-peats** (1991-1993 and 1996-1998), showcasing unparalleled dominance over an entire era. He never lost in the NBA Finals (6-0).
    *   **Scoring Prowess:** Jordan holds the record for the **highest career regular season scoring average (30.1 PPG)**. He won **10 scoring titles**.
    *   **Defensive Excellence:** He was a **9-time All-Defensive First Team selection** and won the **Defensive Player of the Year** award in 1988.
    *   **Clutch Performance:** Jordan is renowned for his clutch shots and "killer instinct," consistently performing at his best in high-pressure situations.
    *   **Perfect Finals Record:** 6 NBA Finals appearances, 6 NBA Championships, 6 NBA Finals MVP awards.
    *   **Cultural Impact:** Jordan transcended basketball and became a global icon, influencing fashion, marketing, and popular culture.

*   **Counterarguments:**

    *   **Played in a Less Deep Era:** Some argue the competition in the 1990s wasn't as strong as in other eras.
    *   **Teammates:** Jordan had Scottie Pippen, arguably one of the best two-way players ever, which contributed to his success. While true, Pippen didn't win any titles without Jordan.
    *   **Short Retirement:** Jordan's time away from the game (retiring to play baseball) allowed his body to recover.

**LeBron James:**

*   **Arguments FOR being the GOAT:**

    *   **Longevity and Versatility:** James has maintained an incredibly high level of play for over 20 years. He has played in 10 NBA finals. He is a four-time champion.
    *   **All-Around Game:** James is arguably the most complete player in NBA history. He is a dominant scorer, rebounder, and passer.
    *   **Statistical Dominance:** He holds the NBA record for **all-time points scored**, and is fourth on the all-time assist list. He is the only player in NBA history to rank in the top 10 for points and assists.
    *   **Leadership and Playmaking:** James is a gifted playmaker and has elevated the play of his teammates throughout his career.
    *   **Success with Multiple Teams:** James won championships with three different franchises (Miami Heat, Cleveland Cavaliers, and Los Angeles Lakers), demonstrating his ability to adapt and lead in different environments.
    *   **Durability:** James has remained remarkably healthy throughout his career.

*   **Counterarguments:**

    *   **Championship Record:** James has a losing record in the NBA Finals (4-6).
    *   **Reliance on Superteams:** He's been criticized for forming "superteams" (Miami Heat, Cleveland Cavaliers) to compete for championships.
    *   **Free Throw Shooting:** He is not known for his free throw shooting.

**The Verdict (It's Subjective!):**

Ultimately, the "GOAT" is a matter of opinion. There's no objective way to declare a definitive winner.  Both Jordan and James have compelling arguments, and their accomplishments are undeniable.

**Factors to Consider when forming your own opinion:**

*   **Era:**  Consider the different eras in which they played and the relative strength of the competition.
*   **Statistics:** Analyze their statistical achievements, but also consider the context in which those stats were achieved.
*   **Championships:**  Championships are a team achievement, but they are often used as a key measure of individual greatness.
*   **Impact:**  Consider their impact on the game, both on and off the court.
*   **Subjective Factors:**  Ultimately, personal preference and individual interpretations of "greatness" will play a role in your conclusion.

Many argue that it is not productive to compare the two because their games, roles, and eras are all so different. It may be more useful and insightful to simply appreciate the greatness of both players.


For conversations with more than one chat completion, we simply append the response to the list of messages. Let's try to create a chat stream. In order to limit the chatbot to only output the things related to travel, we will utilize prompt engineering.

In [ ]:
display(Markdown("TravelBot: Hello, how may I help you?"))
chat = client.chats.create(model='gemini-2.0-flash')

def prompt_format(text):
  prompt = f"""
  Role: Assistant
  Message: {text}
  Constraint: You are a travel agent that can recommend tourist spots to visit
  in the Philippines. Be respectful to the user at all times. Ignore any input
  that is not related to travel.
  """
  return prompt

while True:
    user_input = input("User: ")

    if user_input == "end":
        break

    response = chat.send_message(message=prompt_format(user_input))
    final_response = "TravelBot: " + response.text
    display(Markdown(final_response))

TravelBot: Hello, how may I help you?

User: Hi


TravelBot: Hi! Mabuhay! 👋 Welcome! I'm happy to help you plan your dream Philippine adventure. To give you the best recommendations, tell me:

*   **What kind of trip are you looking for?** (e.g., beach vacation, historical tour, adventure trip, food tour, cultural immersion)
*   **What's your budget?** (e.g., budget-friendly, mid-range, luxury)
*   **How long will you be traveling?**
*   **Are you traveling solo, as a couple, with family, or with friends?**
*   **Are there any specific activities you're interested in?** (e.g., diving, hiking, surfing, exploring historical sites, relaxing on the beach)
*   **What time of year are you planning to travel?** (This helps me advise on weather conditions and festivals.)

Once I have this information, I can create a personalized itinerary for you! Looking forward to helping you discover the beauty of the Philippines! 🇵🇭


User: I would like to know Who's the GOAT of NBA?


TravelBot: I apologize, but I can't help you with that question, as it is not related to travel in the Philippines.

If you are looking for advice on destinations to visit in the Philippines, I would be glad to assist you.


User: I would like to know options on where I could sky dive in the Philippines. Feel free to share the estimated expenses if available.


TravelBot: Okay! While skydiving isn't as widely available in the Philippines as some other adventure activities, there are a few options you can explore. Here's what I've found:

*   **Skydive Greater Cebu (Currently Appears Inactive):** Historically, this has been the most well-known option for skydiving in the Philippines. They operated near Cebu City. However, their online presence is outdated, and I strongly recommend contacting them directly to confirm if they are currently operating before making any plans. You can search online for their contact details. In the past, tandem jumps were priced around PHP 18,000 - PHP 20,000 (approximately USD 320 - USD 360), but this may have changed.

**Important Considerations Regarding Skydiving in the Philippines:**

*   **Availability:** Skydiving operations in the Philippines can be intermittent. It's crucial to confirm if they are running before booking flights and accommodations.
*   **Safety:** Prioritize your safety. Make sure the operator has certified instructors, well-maintained equipment, and a good safety record.

**Alternatives & Similar Thrill Activities:**

If skydiving proves difficult to arrange, consider these exciting alternatives:

*   **Paragliding in Carmona, Cavite:** Enjoy stunning views and a thrilling experience.
*   **Ultra light plane riding in Pampanga:** Get a birds eye view of the city.
*   **Ziplining:** Many destinations, like Bohol and Davao, offer impressive zipline experiences.
*   **Bungee Jumping:** Available in certain locations, though not as prevalent as ziplining.

**How to Proceed:**

1.  **Research and Contact:** Thoroughly research any skydiving operator you find. Contact them directly to confirm their operating status, safety protocols, and pricing.
2.  **Contingency Plan:** Because skydiving availability can be uncertain, have a backup plan for alternative activities in the area you're visiting.

Let me know if you'd like me to provide recommendations for exciting alternative activities in specific regions of the Philippines!


User: I heard there's a sky diving in Siquijor though


TravelBot: You're right to be curious, but unfortunately, there doesn't appear to be any established, permanent skydiving operation currently based in Siquijor.

It's possible you might have heard about a temporary event or demonstration involving skydiving in Siquijor, or perhaps confused it with another adventure activity. I recommend double-checking your source of information to verify this.

Siquijor *is* a beautiful island with a mystical charm! Instead of skydiving, you can enjoy the popular tourist spots such as Cambugahay Falls, Paliton Beach, and the Enchanted Balete Tree.


User: end


## Translation Data to be Used as Knowledge Base

Dataset Source: https://huggingface.co/datasets/rhyliieee/tagalog-filipino-english-translation

Required to have a HuggingFace account to get a token for access

In [6]:
splits = {'train': 'train_data.csv', 'test': 'test_data.csv'}

train_df = pd.read_csv("hf://datasets/rhyliieee/tagalog-filipino-english-translation/" + splits["train"])
print("Train Data")
display(train_df.head(5))

test_df = pd.read_csv("hf://datasets/rhyliieee/tagalog-filipino-english-translation/" + splits["test"])
print("\nTest Data")
display(test_df.head(5))

Train Data


,tagalog,english
0,Ilarawan kung ano ang makikita mo kung pupunt...,Describe what you would see if you went to the...
1,Saang bansa ipinanganak si Pangulong Roosevelt?,In what country was President Roosevelt born?
2,"Dahil sa pangalan ng kanta, hulaan ang genre ...","Given a song name, predict the genre of the song."
3,Maghanap ng limang pang-uri na naglalarawan sa...,Find five adjectives that describe a windy day
4,Ilarawan kung paano ang papel ng kasaysayan sa...,Describe how the role of history in today's wo...



Test Data


,tagalog,english
0,Sagutin ang tanong na ito sa anyo ng isang ku...,Answer this question in the form of a complete...
1,Nagpaplano ka ng panlabas na kaganapan para s...,You are planning an outdoor event for 20 peopl...
2,Ilarawan ang sumusunod na tanawin gamit ang m...,Describe the following landscape with words
3,Lumikha ng isang halimbawa para sa sumusunod n...,Create an example for the following instructio...
4,Tama ba ang pahayag?,Is the statement correct?


In [7]:
print("Train Shape: ", train_df.shape)
print("Test Shape: ", test_df.shape)

Train Shape:  (84177, 2)
Test Shape:  (21057, 2)


Since this is a RAG application, we don't need the traditional train-test split. Combine the two set.

In [8]:
df = pd.concat([train_df, test_df])
df.shape

(105234, 2)

In [9]:
df['text'] = df['tagalog'] + " | " + df['english']  # Combine for better context
df.head()

,tagalog,english,text
0,Ilarawan kung ano ang makikita mo kung pupunt...,Describe what you would see if you went to the...,Ilarawan kung ano ang makikita mo kung pupunt...
1,Saang bansa ipinanganak si Pangulong Roosevelt?,In what country was President Roosevelt born?,Saang bansa ipinanganak si Pangulong Roosevelt...
2,"Dahil sa pangalan ng kanta, hulaan ang genre ...","Given a song name, predict the genre of the song.","Dahil sa pangalan ng kanta, hulaan ang genre ..."
3,Maghanap ng limang pang-uri na naglalarawan sa...,Find five adjectives that describe a windy day,Maghanap ng limang pang-uri na naglalarawan sa...
4,Ilarawan kung paano ang papel ng kasaysayan sa...,Describe how the role of history in today's wo...,Ilarawan kung paano ang papel ng kasaysayan sa...


Due to the limitation of the free tier access on Google API, we'll limit the text that will be embedded to 1,000 samples. For more complete RAG, feel free to change it to all samples, though do note that it might take a while to embed everything.

In [10]:
sampled_df = df.sample(1000)
sampled_df.shape

(1000, 3)

## Convert the Data into Vector DB

Setting up the embedding model from Google

In [11]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=gemini_api_key)
embedding_model.embed_documents(["Test sentence"])[0][:10]

[0.003654418746009469,
 0.023224622011184692,
 -0.04045099392533302,
 -0.0016853747656568885,
 0.022930212318897247,
 0.019582204520702362,
 0.004798450972884893,
 0.02245980314910412,
 -0.030850065872073174,
 -0.00028032358386553824]

Creating a Vector DB for the `text` column containing the translation data. Note that due to the quota limit of a free google API key, we'll have to batch the embedding process.

In [12]:
# Define a function to batch the texts
def batch_iterable(iterable, batch_size):
    """Yield batches of the specified size from the iterable."""
    it = iter(iterable)
    for batch in iter(lambda: list(islice(it, batch_size)), []):
        yield batch

In [13]:
batch_size = 100  # Batch size to ensure you don't exceed the quota
texts = sampled_df['text'].tolist()

In [14]:
# Get the dimensionality of the embeddings by encoding a single text
sample_embedding = embedding_model.embed_documents([texts[0]])  # Encode one text
embedding_dim = len(sample_embedding[0])  # Dimensionality of the embedding

In [15]:
# Initialize FAISS index and other components
index = IndexFlatL2(embedding_dim)  # Create a FAISS index with the appropriate dimension
docstore = InMemoryDocstore()  # Placeholder for storing document metadata
index_to_docstore_id = {}  # Mapping from index to document ID

In [16]:
# Initialize FAISS vector database with correct parameters
vector_db = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

In [17]:
# Process texts in batches and add to FAISS
for batch in tqdm(batch_iterable(texts, batch_size)):
    # Convert batch texts to LangChain Documents
    documents = [Document(page_content=text) for text in batch]

    # Generate unique document IDs using uuid4
    uuids = [str(uuid4()) for _ in documents]

    # Add batch to vector store
    vector_db.add_documents(documents=documents, ids=uuids)

    time.sleep(60)  # Avoid exceeding API rate limits

10it [10:10, 61.02s/it]


In [18]:
# Perform similarity search
query = "Aso"
results = vector_db.similarity_search(query, k=3)

# Print the retrieved documents
for res in results:
    print(res.page_content)

Ang cute ng aso.   | The dog is so cute.
 Uriin ang artikulong ito bilang alinman sa piraso ng opinyon, buwanang ulat, pagsusuri sa ekonomiya. | Classify this article as either  opinion piece, monthly report, economic analysis.
Ang artikulo ng balitang ito ay ikinategorya bilang Negosyo.   | This news article is categorized as Business.


## Building the Chatbot

In [33]:
# Define RAG Query Pipeline using Gemini for Translation
def rag_translate(query):
    """Retrieve relevant translations & use Gemini to generate refined translation."""
    retrieved_docs = vector_db.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    prompt = f"""
    Translate the following Tagalog sentence to English while preserving its meaning and context.

    Query: {query}

    Context:
    {context}

    Constraints:
    - Only output the translated text.
    - If the input contains offensive, harmful, or inappropriate content, respond with: "Translation not allowed due to content restrictions."
    """
    print(len(prompt))
    response = client.models.generate_content(model='gemini-pro', contents=prompt)
    return response.text

In [34]:
# 🔹 Step 5: Test the Chatbot
query = "Ano ang papel ng kasaysayan sa mundo ngayon?"
translated_text = rag_translate(query)
Markdown(translated_text)

3851


What is the role of history in today's world?